# This is currently a work in progress.  Feel free to add to it as it progresses along!!!

## This notebook is a gradual work in progress to make this data set more complete and easier for people to import and use for their own data analysis with minimal data cleaning.

In [1]:
import pandas as pd
import os

## Read in the raw data files and combine into one data frame

In [2]:
file_list = os.listdir('raw_data/')

In [75]:
d = pd.concat([pd.read_csv('raw_data/' + f) for f in file_list], keys=file_list, names = ['FileName','RowNumber'])

In [4]:
df = d.reset_index().drop('RowNumber', axis = 1)

In [5]:
df.head()

,FileName,AdministrativeAdjustmentIndicator,CADAddress,CADStreet,CodedMonth,Complaint,Count,Crime,DateOccur,DateOccured,...,ILEADSAddress,ILEADSStreet,LocationComment,LocationName,MonthReportedtoMSHP,Neighborhood,NewCrimeIndicator,UnfoundedCrimeIndicator,XCoord,YCoord
0,april2008.csv,,4900,MAPLE,NaN,NaN,1,177000,NaN,01/01/2000 20:58,...,4900,MAPLE AVE,,,2008-04,53,Y,,888708.3,1028833.0
1,april2008.csv,,4357,CALIFORNIA,NaN,NaN,1,67501,NaN,01/01/2003 00:01,...,4357,CALIFORNIA AVE,,,2008-04,17,Y,,896256.1,998745.9
2,april2008.csv,,4313,MARGARETTA,NaN,NaN,1,115400,NaN,01/01/2004 00:01,...,4313,MARGARETTA AVE,,,2008-04,68,Y,,896282.4,1033555.0
3,april2008.csv,,3939,OHIO,NaN,NaN,1,115400,NaN,01/01/2006 10:00,...,3939,OHIO AVE,,,2008-04,16,Y,,897347.3,1001419.0
4,april2008.csv,,2612,ANNIE MALONE DR,NaN,NaN,1,91113,NaN,01/01/2007 16:36,...,2612,ANNIE MALONE DR,@ANNIE MALONE HOME,,2008-04,57,Y,,895420.8,1029119.0


## Load in neighborhood lookup table and join it with the data frame

In [52]:
# Load in neighborhood lookup table
nbhds = pd.read_csv('neighborhood_lookup.csv')

In [53]:
df2 = pd.merge(df, nbhds, left_on = 'Neighborhood', right_on = 'Neighborhood Number', how = 'left')

## Fixing column names to something more consistent

In [54]:
col_names = list(df2.columns)
df2.columns = [name.replace(' ', '') for name in col_names]

In [55]:
df2.columns

Index([u'FileName', u'AdministrativeAdjustmentIndicator', u'CADAddress', u'CADStreet', u'CodedMonth', u'Complaint', u'Count', u'Crime', u'DateOccur', u'DateOccured', u'Description', u'District', u'FlagAdministrative', u'FlagCleanup', u'FlagCrime', u'FlagUnfounded', u'ILEADSAddress', u'ILEADSStreet', u'LocationComment', u'LocationName', u'MonthReportedtoMSHP', u'Neighborhood', u'NewCrimeIndicator', u'UnfoundedCrimeIndicator', u'XCoord', u'YCoord', u'NeighborhoodNumber', u'NeighborhoodName', u'NeighborhoodPrimaryDistrict', u'NeighborhoodAddlDistrict'], dtype='object')

## Working to fill in missing data

In [56]:
df2['AdministrativeAdjustmentIndicator'] = df2['AdministrativeAdjustmentIndicator'].replace(' ', pd.np.NaN)

## Here we're trying to align the crime codes in the Crime field to the Uniform Crime Reporting codes.
First we need to pad out the Crime column to be six digits, since the leading zero was trimmed in the .csv files

In [92]:
df2['Crime'] = df2['Crime'].astype(str)
df2['Crime'] = df2['Crime'].apply(lambda x: x.zfill(6))

## Write the file out to tab delimited file

In [95]:
df2.to_csv('clean_data/stl_crime_data.txt', sep='\t')